# Grid Functions and Grid Parameters

## Author: Zach Etienne

## This notebook explores how NRPy registers grid parameters and grid functions across multiple infrastructures. It walks through the high-level Python interfaces in `grid.py`, showing how to create scalar, vector, and tensor gridfunctions, how to inspect their metadata, and how to obtain C-style access strings for different infrastructures (BHaH, ETLegacy, and CarpetX). It also demonstrates parity handling, group-based `#define` generation, higher-rank tensor registration, and a small worked finite difference kernel using these tools.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, you can treat this as a hands-on reference; you should be able to pick up the syntax by following the examples.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**

### NRPy Source Code for this module:  
* [grid.py](../edit/grid.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python/NRPy-grid-modules): Initialize core Python/NRPy grid modules
1. [Step 2](#Step-2:-Register-basic-scalar-gridfunctions): Register basic scalar gridfunctions
1. [Step 3](#Step-3:-Register-vector-and-tensor-gridfunctions-(rank-1-and-rank-2)): Register vector and tensor gridfunctions (rank 1 and rank 2)
1. [Step 4](#Step-4:-Inspecting-gridfunction-metadata-and-groups): Inspecting gridfunction metadata and groups
1. [Step 5](#Step-5:-Accessing-gridfunction-data-from-memory-in-different-infrastructures): Accessing gridfunction data from memory in different infrastructures
1. [Step 6](#Step-6:-Handling-parity-types-for-gridfunctions): Handling parity types for gridfunctions
1. [Step 7](#Step-7:-Working-with-gridfunction-defines-and-parameter-arrays): Working with gridfunction defines and parameter arrays
1. [Step 8](#Step-8:-Higher-rank-tensors-and-symmetries): Higher rank tensors and symmetries
1. [Step 9](#Step-9:-Putting-it-all-together:-a-mini-finite-difference-kernel): Putting it all together: a mini finite difference kernel

# Step 1: Initialize core Python/NRPy grid modules
### [Back to [top](#Table-of-Contents)]

We start by importing the core pieces needed to work with grid parameters and gridfunctions:

* `sympy` for symbolic expressions representing gridfunctions.
* `nrpy.params` (`par`) for infrastructure and parameter control.
* `nrpy.grid` (`grid`) for gridfunction registration and utilities.

The `Infrastructure` parameter is central here; it controls which gridfunction subclass is used internally:

* `"BHaH"`: BlackHoles@Home infrastructure.
* `"ETLegacy"`: classic Einstein Toolkit gridfunctions.
* `"CarpetX"`: CarpetX based gridfunctions.

The `grid.py` module may also register a few global grid parameters, such as:

* `Cart_originx`, `Cart_originy`, `Cart_originz`: Cartesian coordinates of the grid origin.
* `NUMGRIDS`: number of grids in the hierarchy.
* `grid_rotates`: boolean flag indicating whether the grid frame rotates.
* `MAXNUMGRIDS`: compile-time cap on the number of grids.

Different NRPy versions may or may not register these automatically at import time, so we will query them defensively.

In [1]:
# Step 1: Initialize core Python/NRPy grid modules
import sympy as sp
import nrpy.params as par
import nrpy.grid as grid

# Always start with a clean registry of gridfunctions for tutorial experiments
grid.glb_gridfcs_dict.clear()

# Set a default infrastructure. We will change this later as needed.
par.set_parval_from_str("Infrastructure", "BHaH")

print("Infrastructure:", par.parval_from_str("Infrastructure"))

# Safely query grid-related parameters, which may or may not be registered
param_names = ["Cart_originx", "Cart_originy", "Cart_originz",
               "NUMGRIDS", "grid_rotates", "MAXNUMGRIDS"]

print("\nGrid related parameters (if registered):")
for name in param_names:
    try:
        val = par.parval_from_str(name)
    except ValueError:
        val = "(not registered in this NRPy build)"
    print(f"  {name}: {val}")

Infrastructure: BHaH

Grid related parameters (if registered):
  Cart_originx: (not registered in this NRPy build)
  Cart_originy: (not registered in this NRPy build)
  Cart_originz: (not registered in this NRPy build)
  NUMGRIDS: (not registered in this NRPy build)
  grid_rotates: (not registered in this NRPy build)
  MAXNUMGRIDS: (not registered in this NRPy build)


If your version of `grid.py` exposes these parameters, you can adjust them at runtime, for example:

```python
par.set_parval_from_str("Cart_originx", 10.0)
par.parval_from_str("Cart_originx")
```

Gridfunctions defined in later steps implicitly live on the grids controlled by these parameters.

# Step 2: Register basic scalar gridfunctions
### [Back to [top](#Table-of-Contents)]

The main scalar entry point into `grid.py` is the `register_gridfunctions()` function. It:

* Registers one or more gridfunctions in the global dictionary `grid.glb_gridfcs_dict`.
* Returns SymPy symbols that you can use in algebraic expressions.
* Constructs infrastructure specific `GridFunction` objects that carry metadata (group, rank, dimension, etc).

The simplest call looks like:

```python
sym = grid.register_gridfunctions("u")[0]
```

This:

* Registers a scalar, rank 0 gridfunction named `"u"` in the default group `"EVOL"`.
* Returns a SymPy symbol `$u$` that you can use in expressions such as $u^2$.

In [2]:
# Step 2: Register basic scalar gridfunctions

# Work with a clean registry for this step
grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Register a single scalar gridfunction named "u"
u = grid.register_gridfunctions("u")[0]
print("SymPy symbol for u:", u)

# Register multiple scalar gridfunctions at once
rho, P = grid.register_gridfunctions(["rho", "P"])
print("SymPy symbols for multiple scalars:", rho, P)

# Control additional properties such as f_infinity and wavespeed
phi = grid.register_gridfunctions("phi", f_infinity=0.0, wavespeed=1.0)[0]
print("SymPy symbol for phi:", phi)

# A small symbolic expression to show these behave like ordinary SymPy symbols
expr = rho * u**2 + P * phi
print("\nExample SymPy expression:")
print("expr =", expr)

# Inspect what keys ended up in the global gridfunction dictionary
print("\nRegistered gridfunctions (names only):")
for name in sorted(grid.glb_gridfcs_dict):
    print("  ", name)

SymPy symbol for u: u
SymPy symbols for multiple scalars: rho P
SymPy symbol for phi: phi

Example SymPy expression:
expr = P*phi + rho*u**2

Registered gridfunctions (names only):
   P
   phi
   rho
   u


Things to notice:

* The return value from `register_gridfunctions()` is always a list of SymPy symbols, even when you provide a single name.
* All registered gridfunctions are tracked in `grid.glb_gridfcs_dict`, which maps names (for example `"u"`) to `GridFunction` objects.
* Gridfunctions are real by default, so you can safely use them in real valued expressions such as $u^2$ or $\\rho P$.

In the next step we use specialized helpers to create vector and tensor gridfunctions.

# Step 3: Register vector and tensor gridfunctions (rank 1 and rank 2)
### [Back to [top](#Table-of-Contents)]

While `register_gridfunctions()` is enough for standalone scalars, vector and tensor quantities are more naturally created with:

* `register_gridfunctions_for_single_rank1()` for rank 1 tensors (vectors),
* `register_gridfunctions_for_single_rank2()` for rank 2 tensors (matrices),
* and the fully general `register_gridfunctions_for_single_rankN()` for rank $N$ tensors.

These helpers:

* Declare indexed SymPy objects using NRPy indexed expression utilities.
* Generate component names like `"betU0"`, `"betU1"`, `"betU2"` for a rank 1 vector `betU`.
* Register one `GridFunction` per unique component name.

In this step we:

1. Register a 3 component vector field `betU`.
2. Register a symmetric $3 \times 3$ tensor field `gDD`.
3. Build a small symbolic expression that uses these components.

In [3]:
# Step 3: Register vector and tensor gridfunctions (rank 1 and rank 2)

grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Rank 1 example: a 3-component vector betU^i
betU = grid.register_gridfunctions_for_single_rank1("betU")  # default dimension=3
print("Rank 1 vector components (SymPy):")
print(betU)

# Rank 2 example: a symmetric 3x3 tensor gDD_ij
gDD = grid.register_gridfunctions_for_single_rank2("gDD", symmetry="sym01")
print("\nRank 2 symmetric tensor components (as a 2D list):")
for row in gDD:
    print(row)

# Use these in a simple symbolic expression
# Example: trace of gDD_ij times betU^i betU^j (a toy quadratic form)
trace_g = gDD[0][0] + gDD[1][1] + gDD[2][2]
quad_form = 0
for i in range(3):
    for j in range(3):
        quad_form += gDD[i][j] * betU[i] * betU[j]

print("\nExample expressions:")
print("trace_g =", trace_g)
print("quad_form =", quad_form)

print("\nRegistered component names in glb_gridfcs_dict:")
for name in sorted(grid.glb_gridfcs_dict):
    print("  ", name)

Rank 1 vector components (SymPy):
[betU0, betU1, betU2]

Rank 2 symmetric tensor components (as a 2D list):
[gDD00, gDD01, gDD02]
[gDD01, gDD11, gDD12]
[gDD02, gDD12, gDD22]

Example expressions:
trace_g = gDD00 + gDD11 + gDD22
quad_form = betU0**2*gDD00 + 2*betU0*betU1*gDD01 + 2*betU0*betU2*gDD02 + betU1**2*gDD11 + 2*betU1*betU2*gDD12 + betU2**2*gDD22

Registered component names in glb_gridfcs_dict:
   betU0
   betU1
   betU2
   gDD00
   gDD01
   gDD02
   gDD11
   gDD12
   gDD22


Key observations:

* For a rank 1 vector in $3$ dimensions, `register_gridfunctions_for_single_rank1("betU")` returns a Python list `[betU0, betU1, betU2]`.
* For a symmetric rank 2 tensor in $3$ dimensions, `register_gridfunctions_for_single_rank2("gDD", symmetry="sym01")` returns a $3 \times 3$ nested list.
* Each unique component name (for example `"gDD01"`, `"gDD22"`) becomes its own gridfunction in `glb_gridfcs_dict`.

The nested SymPy structure lets you write continuum tensor expressions, while the grid registry keeps track of how each component will be stored and accessed in C.

# Step 4: Inspecting gridfunction metadata and groups
### [Back to [top](#Table-of-Contents)]

Each entry in `glb_gridfcs_dict` is an instance of an infrastructure specific subclass of `GridFunction`. These classes store metadata such as:

* `name`: the component name (for example `"gDD01"`),
* `group`: which gridfunction group it belongs to (`"EVOL"`, `"AUXEVOL"`, `"DIAG"`, `"AUX"`),
* `rank`: tensor rank (0 for scalar, 1 for vector, 2 for matrix, etc),
* `dimension`: spatial dimension (usually $3$ or $4$),
* `gf_type`: C type used in the underlying infrastructure (for example `"REAL"` or `"CCTK_REAL"`),
* `f_infinity`: asymptotic value at spatial infinity,
* `wavespeed`: characteristic wave speed.

There is also a notion of **grouping**, which lets you ask for lists of EVOL, AUXEVOL, DIAG, and AUX gridfunctions.

In this step we:

1. Register gridfunctions in different groups.
2. Inspect the metadata for one of them.
3. Use the group helper `gridfunction_lists()` to organize names.

In [4]:
# Step 4: Inspecting gridfunction metadata and groups

grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Register a mix of gridfunctions in different groups
alpha = grid.register_gridfunctions("alpha", f_infinity=1.0, wavespeed=0.0)[0]
phi = grid.register_gridfunctions("phi", f_infinity=0.0, wavespeed=1.0)[0]
K = grid.register_gridfunctions("K", group="AUXEVOL")[0]
H = grid.register_gridfunctions("H", group="DIAG")[0]
aux_scalar = grid.register_gridfunctions("aux_scalar", group="AUX")[0]

# Pick one gridfunction and inspect its metadata
gf_obj = grid.glb_gridfcs_dict["alpha"]
print("Metadata for gridfunction 'alpha':")
print("  type       :", type(gf_obj).__name__)
print("  name       :", gf_obj.name)
print("  group      :", gf_obj.group)
print("  rank       :", gf_obj.rank)
print("  dimension  :", gf_obj.dimension)
print("  gf_type    :", gf_obj.gf_type)
print("  f_infinity :", gf_obj.f_infinity)
print("  wavespeed  :", gf_obj.wavespeed)

# Use the convenience helper to get lists of names by group
evol_gfs, auxevol_gfs, diag_gfs, aux_gfs = grid.GridFunction.gridfunction_lists()
print("\nGridfunctions by group:")
print("  EVOL    :", evol_gfs)
print("  AUXEVOL :", auxevol_gfs)
print("  DIAG    :", diag_gfs)
print("  AUX     :", aux_gfs)

Metadata for gridfunction 'alpha':
  type       : BHaHGridFunction
  name       : alpha
  group      : EVOL
  rank       : 0
  dimension  : 3
  gf_type    : REAL
  f_infinity : 1.0
  wavespeed  : 0.0

Gridfunctions by group:
  EVOL    : ['alpha', 'phi']
  AUXEVOL : ['K']
  DIAG    : ['H']
  AUX     : ['aux_scalar']


You can choose the group at registration time with the `group` keyword:

```python
v = grid.register_gridfunctions("v", group="AUX")[0]
```

EVOL gridfunctions are typically the ones that appear on the left hand side of evolution equations, AUXEVOL gridfunctions are needed everywhere to update these evolutions, and AUX or DIAG gridfunctions store helper or diagnostic quantities.

# Step 5: Accessing gridfunction data from memory in different infrastructures
### [Back to [top](#Table-of-Contents)]

A core task for any grid based code is converting the abstract idea of a gridfunction evaluated at offsets $(i_{0} + \Delta i_{0}, i_{1} + \Delta i_{1}, i_{2} + \Delta i_{2})$ into a concrete C array access.

`grid.py` provides infrastructure aware helpers for this:

* Each `GridFunction` subclass implements a `read_gf_from_memory_Ccode_onept()` method that returns a C-style string describing how to read a single point from memory.
* Some subclasses also provide static helpers like `access_gf()` to construct these access strings directly.

In this step we:

1. Switch between infrastructures.
2. Register a simple scalar gridfunction.
3. Ask the gridfunction for its C access string at various offsets, with and without SIMD enabled.
4. Inspect some infrastructure specific attributes side-by-side.

In [5]:
# Step 5: Accessing gridfunction data from memory in different infrastructures

def demo_read_gf_onept(infrastructure: str) -> None:
    print("\n=== Infrastructure:", infrastructure, "===")
    par.set_parval_from_str("Infrastructure", infrastructure)
    grid.glb_gridfcs_dict.clear()

    # Register a single scalar gridfunction
    u = grid.register_gridfunctions("u")[0]
    gf_obj = grid.glb_gridfcs_dict["u"]

    # Plain access at the current grid point
    c_expr_center = gf_obj.read_gf_from_memory_Ccode_onept(0, 0, 0)
    print("Center access:          ", c_expr_center)

    # Offset access one point in x and -2 in z
    c_expr_offset = gf_obj.read_gf_from_memory_Ccode_onept(1, 0, -2)
    print("Offset access (1,0,-2):", c_expr_offset)

    # SIMD aware access
    c_expr_simd = gf_obj.read_gf_from_memory_Ccode_onept(0, 0, 0, enable_simd=True)
    print("SIMD access:            ", c_expr_simd)

    # A quick look at infrastructure specific details
    print("Underlying Python class:", type(gf_obj).__name__)
    print("gf_type:", gf_obj.gf_type)
    print("Object attributes:", sorted(gf_obj.__dict__.keys()))


# BHaH infrastructure
demo_read_gf_onept("BHaH")

# ETLegacy infrastructure
demo_read_gf_onept("ETLegacy")

# CarpetX infrastructure
demo_read_gf_onept("CarpetX")


=== Infrastructure: BHaH ===
Center access:           in_gfs[IDX4(UGF, i0, i1, i2)]
Offset access (1,0,-2): in_gfs[IDX4(UGF, i0+1, i1, i2-2)]
SIMD access:             ReadSIMD(&in_gfs[IDX4(UGF, i0, i1, i2)])
Underlying Python class: BHaHGridFunction
gf_type: REAL
Object attributes: ['desc', 'dimension', 'f_infinity', 'gf_array_name', 'gf_type', 'group', 'is_basename', 'name', 'rank', 'sync_gf_in_superB', 'wavespeed']

=== Infrastructure: ETLegacy ===
Center access:           uGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]
Offset access (1,0,-2): uGF[CCTK_GFINDEX3D(cctkGH, i0+1, i1, i2-2)]
SIMD access:             ReadSIMD(&uGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)])
Underlying Python class: ETLegacyGridFunction
gf_type: CCTK_REAL
Object attributes: ['desc', 'dimension', 'f_infinity', 'gf_type', 'group', 'is_basename', 'name', 'rank', 'wavespeed']

=== Infrastructure: CarpetX ===
Center access:           uGF(p.I)
Offset access (1,0,-2): uGF(p.I + 1*p.DI[0] - 2*p.DI[2])
SIMD access:             Re

You should see three different families of C-style access strings, for example (exact details may vary):

* For `"BHaH"` you might see patterns like:

  * `in_gfs[IDX4(UGF, i0+1, i1, i2-2)]`

* For `"ETLegacy"`:

  * `uGF[CCTK_GFINDEX3D(cctkGH, i0+1, i1, i2-2)]`

* For `"CarpetX"`:

  * `uGF(p.I + 1*p.DI[0] - 2*p.DI[2])`

The `enable_simd=True` flag usually adds a `ReadSIMD(&...)` wrapper around the access, which connects directly to SIMD load intrinsics in the generated C code.

The side-by-side inspection of `type(gf_obj).__name__`, `gf_type`, and the attribute keys shows how the same high-level registration call produces different infrastructure specific objects.

# Step 6: Handling parity types for gridfunctions
### [Back to [top](#Table-of-Contents)]

Many codes enforce symmetry at boundaries using parity conditions. `grid.py` supports this by associating an integer parity code with each gridfunction component.

The main tools are:

* `GridFunction.get_parity_type(name, rank, dimension)`:
  * Computes a parity code for a single component name such as `"phi"` or `"gDD12"`.
* `GridFunction.set_parity_types(list_of_gf_names)`:
  * Looks up each name in `glb_gridfcs_dict`,
  * inspects the registered `rank` and `dimension`,
  * and returns a list of parity codes.

The exact meaning of each integer code is defined by the infrastructure, but these codes can be used consistently to fill parity tables in C.

In this step we:

1. Register a scalar, a vector, and a symmetric tensor.
2. Query their parity types directly.
3. Ask `set_parity_types()` to produce a parity table.

In [6]:
# Step 6: Handling parity types for gridfunctions

grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Register a scalar phi and a vector betU^i and a symmetric tensor gDD_ij
phi = grid.register_gridfunctions("phi")[0]
betU = grid.register_gridfunctions_for_single_rank1("betU")
gDD = grid.register_gridfunctions_for_single_rank2("gDD", symmetry="sym01")

# Example 6a: Query parity type for individual names
print("Individual parity type queries:")
print("  phi   (rank 0):", grid.GridFunction.get_parity_type("phi", rank=0, dimension=3))
print("  betU2 (rank 1):", grid.GridFunction.get_parity_type("betU2", rank=1, dimension=3))
print("  gDD12 (rank 2):", grid.GridFunction.get_parity_type("gDD12", rank=2, dimension=3))

# Example 6b: Let GridFunction.set_parity_types() inspect registered gfs
names_to_check = ["phi", "betU0", "betU1", "betU2", "gDD00", "gDD01", "gDD22"]
parity_codes = grid.GridFunction.set_parity_types(names_to_check)

print("\nParity types determined from registered gridfunctions:")
for name, code in zip(names_to_check, parity_codes):
    print(f"  {name:6s} -> parity_type = {code}")

Individual parity type queries:
  phi   (rank 0): 0
  betU2 (rank 1): 3
  gDD12 (rank 2): 8

Parity types determined from registered gridfunctions:
  phi    -> parity_type = 0
  betU0  -> parity_type = 1
  betU1  -> parity_type = 2
  betU2  -> parity_type = 3
  gDD00  -> parity_type = 4
  gDD01  -> parity_type = 5
  gDD22  -> parity_type = 9


Notes:

* Scalars return a single parity code because they have a simple behavior under spatial inversion.
* For vectors and tensors, component names such as `"betU2"` or `"gDD12"` encode their spatial indices. The helper functions decode the final digits and map them to parity codes using dimension dependent lookup tables.
* `GridFunction.set_parity_types()` operates on registered gridfunctions, so it can use the stored `rank` and `dimension` values instead of requiring you to pass them explicitly.

These parity codes can then be written into arrays in C and used to impose symmetry at boundaries.

# Step 7: Working with gridfunction defines and parameter arrays
### [Back to [top](#Table-of-Contents)]

For the BHaH infrastructure, `grid.py` can generate C `#define` blocks that:

* Assign a unique integer index to each gridfunction in each group (for example `"UGF"`, `"PHIGF"`, and so on).
* Provide arrays that store $f_{\infty}$ (value at spatial infinity) and characteristic wave speeds for evolved gridfunctions.

This is handled by `BHaHGridFunction.gridfunction_defines()`, which returns a string ready to be written into a C header.

In this step we:

1. Register a few evolved and auxiliary gridfunctions.
2. Call `grid.BHaHGridFunction.gridfunction_defines()`.
3. Inspect the generated C-style text alongside the grid parameters from Step 1.

In [7]:
# Step 7: Working with gridfunction defines and parameter arrays

grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Register evolved gridfunctions with different f_infinity and wavespeed values
alpha = grid.register_gridfunctions("alpha", f_infinity=1.0, wavespeed=0.0)[0]
phi = grid.register_gridfunctions("phi", f_infinity=0.0, wavespeed=1.0)[0]
Pi = grid.register_gridfunctions("Pi", f_infinity=0.0, wavespeed=1.0)[0]

# Register an AUXEVOL and an AUX gridfunction
K = grid.register_gridfunctions("K", group="AUXEVOL")[0]
aux_scalar = grid.register_gridfunctions("aux_scalar", group="AUX")[0]

# Generate the defines string for the current set of registered gridfunctions
defines_str = grid.BHaHGridFunction.gridfunction_defines()
print("Generated C-style defines and parameter arrays:\n")
print(defines_str)

# Safely echo grid-related parameters, if available
print("Grid related parameters (if registered):")
for name in ["Cart_originx", "Cart_originy", "Cart_originz", "NUMGRIDS", "grid_rotates"]:
    try:
        val = par.parval_from_str(name)
    except ValueError:
        val = "(not registered in this NRPy build)"
    print(f"  {name}: {val}")

Generated C-style defines and parameter arrays:


// EVOL VARIABLES:
#define NUM_EVOL_GFS 3
#define ALPHAGF	0
#define PHIGF	1
#define PIGF	2


// SET gridfunctions_f_infinity[i] = evolved gridfunction i's value in the limit r->infinity:
static const REAL gridfunctions_f_infinity[NUM_EVOL_GFS] = { 1.0, 0.0, 0.0 };


// SET gridfunctions_wavespeed[i] = evolved gridfunction i's characteristic wave speed:
static const REAL gridfunctions_wavespeed[NUM_EVOL_GFS] = { 0.0, 1.0, 1.0 };

// AUXEVOL VARIABLES:
#define NUM_AUXEVOL_GFS 1
#define KGF	0

// AUX VARIABLES:
#define NUM_AUX_GFS 1
#define AUX_SCALARGF	0

Grid related parameters (if registered):
  Cart_originx: (not registered in this NRPy build)
  Cart_originy: (not registered in this NRPy build)
  Cart_originz: (not registered in this NRPy build)
  NUMGRIDS: (not registered in this NRPy build)
  grid_rotates: (not registered in this NRPy build)


The generated text typically contains:

* `NUM_EVOL_GFS`, `NUM_AUXEVOL_GFS`, and `NUM_AUX_GFS` counts.
* `#define` lines such as `#define ALPHAGF 0`, `#define PHIGF 1`, and so on.
* Two constant arrays:
  * `gridfunctions_f_infinity[NUM_EVOL_GFS]`,
  * `gridfunctions_wavespeed[NUM_EVOL_GFS]`,
  each filled from the `f_infinity` and `wavespeed` attributes of the registered evolved gridfunctions.

This layout is convenient when building C kernels that loop over gridfunctions using integer indices.

# Step 8: Higher rank tensors and symmetries
### [Back to [top](#Table-of-Contents)]

`register_gridfunctions_for_single_rankN()` is the most general helper in `grid.py`. It can:

* Declare rank $N$ indexed SymPy objects with optional symmetries.
* Register one gridfunction per unique component name.
* Return a nested Python list with the same shape as the tensor indices.

For example, a rank 4 tensor such as a Riemann like tensor $R_{abcd}$ in $4$ dimensions can be registered with symmetries built in:

* Symmetry in the first index pair $(a,b)$,
* Symmetry in the second index pair $(c,d)$,
* Additional patterns specified through a symmetry string like `"sym01_sym23"`.

In this step we:

1. Register a rank 4 tensor in $4$ dimensions with pairwise symmetries.
2. Inspect the nested SymPy structure.
3. Look at a sample of registered component names.

In [8]:
# Step 8: Higher rank tensors and symmetries

grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "ETLegacy")

# Register a rank 4 tensor R_abcd with symmetries in (0,1) and (2,3) in 4D
R = grid.register_gridfunctions_for_single_rankN(
    "R", rank=4, symmetry="sym01_sym23", dimension=4, desc="Riemann_tensor"
)

print("Shape hint for rank-4 tensor R_abcd in 4D:")
print("  Number of indices along a:", len(R))
print("  Number of indices along b:", len(R[0]))
print("  Number of indices along c:", len(R[0][0]))
print("  Number of indices along d:", len(R[0][0][0]))

print("\nSample components:")
print("  R[0][0][0][0] =", R[0][0][0][0])
print("  R[0][0][0][1] =", R[0][0][0][1])
print("  R[1][2][3][3] =", R[1][2][3][3])

print("\nTotal number of registered gridfunction components:", len(grid.glb_gridfcs_dict))
print("First 12 component names in sorted order:")
for name in sorted(grid.glb_gridfcs_dict)[:12]:
    print("  ", name)

Shape hint for rank-4 tensor R_abcd in 4D:
  Number of indices along a: 4
  Number of indices along b: 4
  Number of indices along c: 4
  Number of indices along d: 4

Sample components:
  R[0][0][0][0] = R0000
  R[0][0][0][1] = R0001
  R[1][2][3][3] = R1233

Total number of registered gridfunction components: 100
First 12 component names in sorted order:
   R0000
   R0001
   R0002
   R0003
   R0011
   R0012
   R0013
   R0022
   R0023
   R0033
   R0100
   R0101


Observations:

* The returned object `R` is a nested Python list with four indices. Accessing `R[i][j][k][l]` gives the SymPy symbol associated with that component.
* Due to symmetries, many entries reference the same underlying SymPy symbol and gridfunction, which reduces storage and work.
* The `desc` parameter is automatically extended by appending component names so that each registered gridfunction has a descriptive string.

The same function works for rank $1$, $2$, $3$, and $4$, making it the main entry point for complicated tensor fields.

# Step 9: Putting it all together: a mini finite difference kernel
### [Back to [top](#Table-of-Contents)]

Finally, combine several ideas:

* Registering evolved and auxiliary gridfunctions.
* Generating BHaH style access strings.
* Assembling a small C code snippet for a centered finite difference derivative.

We will:

1. Use the BHaH infrastructure.
2. Register an evolved field `u` and an auxiliary evolution field `rhs_u`.
3. Use `BHaHGridFunction.access_gf()` to build C expressions for $u$ at $(i_{0}-1, i_{0}, i_{0}+1)$.
4. Assemble a one line C statement for a centered derivative in the $x$ direction:

$$\partial_{x} u \approx \frac{u_{i_{0}+1} - u_{i_{0}-1}}{2 \Delta x}.$$

In [9]:
# Step 9: Putting it all together: a mini finite difference kernel

grid.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Register one evolved and one AUXEVOL gridfunction
u = grid.register_gridfunctions("u", f_infinity=0.0, wavespeed=1.0)[0]
rhs_u = grid.register_gridfunctions("rhs_u", group="AUXEVOL")[0]

# Build C-style access strings for u(i0-1), u(i0), u(i0+1) along x
u_minus = grid.BHaHGridFunction.access_gf("u", -1, 0, 0, gf_array_name="in_gfs")
u_center = grid.BHaHGridFunction.access_gf("u", 0, 0, 0, gf_array_name="in_gfs")
u_plus = grid.BHaHGridFunction.access_gf("u", 1, 0, 0, gf_array_name="in_gfs")

# And for the RHS storage location (often in a separate AUXEVOL array)
rhs_store = grid.BHaHGridFunction.access_gf(
    "rhs_u", 0, 0, 0, gf_array_name="auxevol_gfs"
)

# Assemble a small C kernel using these access strings
c_kernel_lines = []
c_kernel_lines.append("// Example centered finite difference approximation for du/dx")
c_kernel_lines.append("const REAL inv_2dx = 0.5/dx;")
c_kernel_lines.append(
    f"{rhs_store} = ({u_plus} - {u_minus}) * inv_2dx;  // du/dx at the current point"
)

print("Generated mini C kernel:\n")
print("\n".join(c_kernel_lines))

Generated mini C kernel:

// Example centered finite difference approximation for du/dx
const REAL inv_2dx = 0.5/dx;
auxevol_gfs[IDX4(RHS_UGF, i0, i1, i2)] = (in_gfs[IDX4(UGF, i0+1, i1, i2)] - in_gfs[IDX4(UGF, i0-1, i1, i2)]) * inv_2dx;  // du/dx at the current point


This small example shows how:

* Symbolic registration of gridfunctions leads directly to consistent C access patterns through `grid.py`.
* Group based arrays such as `in_gfs` and `auxevol_gfs` can be combined with offsets to build finite difference stencils.
* The same pattern can be extended to other directions (by changing the offsets and spacing) or to higher order stencils that involve more neighboring points.